In [1]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

In [2]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace,HuggingFaceEmbeddings
from datasets import load_dataset

In [8]:
import cassio

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('budget_speech.pdf')

In [5]:
docs = loader.load()

len(docs)

58

In [10]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="conversational" 
)

model = ChatHuggingFace(llm=llm)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = ""
ASTRA_DB_ID = ""
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [11]:
astra_db_vector = Cassandra(
    embedding=embeddings,
    table_name="QnA",
    session=None,
    keyspace=None
)

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_spliter = RecursiveCharacterTextSplitter(
 chunk_size = 1000   
)


In [13]:
chunks = text_spliter.split_documents(docs)

In [14]:
len(chunks)

122

In [15]:
astra_db_vector.add_documents(chunks)

['f6059a719de34f3cb714eaaa6dda3e2b',
 'd19f05dcfcb34073a028dcf8dedbb812',
 '14c1e1cec102402db468418951615846',
 '296e40ed7ae7481ebbf9dc657075bee8',
 '2b211ed8f98a4246a165e16d808b5923',
 '7594763e45194d599b2396381c75f47d',
 'ffa2c920a3874153a23d583d5dbd5e7b',
 'dbefac2899e34bfeba40c08ac6d3d2e7',
 '2fc38943fab3460f9a29ff980d7175ef',
 '9b48988e350a4e41905158dc22a05772',
 'b4967f41128e495986f6eb525f23b3bf',
 '26cfbab4da8346ecbaa013ce0a4229d6',
 '4b6b297c79b94057a6d2d1992026c9a3',
 '409c6822a18f419084482a73f4d83085',
 '09b4894b3ac346b1bc0d70592db94792',
 '079074424d1547d3b98eac90604d48b4',
 'a5a2bf50f18e4642b92396db4e870324',
 '6e6aa31b510d4fe7beb1504506572ae3',
 '56f49ff2a77d415ca5e3d44aba879958',
 '5f4771bd98e24e59abf54f53a2c82aeb',
 'beea9c0d6efa475bb39b4b86ced46a2d',
 'b731a0d2a7ef472491561330aef7c186',
 '35ef2596ec084159872a1fd78ce8ac5e',
 'bfad4199c9d04f4ca1f97dea47322c89',
 '12af3b94d63145138cdde52647220ab0',
 '15b8f64f6f7643cbaae871674da51013',
 'de8bb07355de41989be486e878b41207',
 

In [16]:
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_db_vector)

In [17]:
astra_vector_index.query("How much the agriculture target will be increased to and what the focus will be",llm=model)

'The agriculture credit target will be increased to ` 20 lakh crore with focus on animal husbandry, dairy and fisheries. '